<a href="https://colab.research.google.com/github/mmistroni/OpenBB_Experiments/blob/main/OpenBBSamples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing the OpenBB Platform in Google Colab

This notebook will install the OpenBB Platform, fetch some data and prepare it for display as a bar chart.

Sign up for a free account here: https://my.openbb.co

In [1]:
# Install the OpenBB Platform with all available extensions.
# Messages indicating package version conflicts at the end of installation can be safely ignored.

!pip install openbb
!pip install pandas_ta
!pip install openbb-charting
!pip install openbb-finviz
!pip install openbb-yfinance
# There is also a nightly distribution available, openbb-nightly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 282.7/282.7 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.0/97.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.7/124.7 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.8/105.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.3/108.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.7/113.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.3/240.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 109.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 11.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.43.0
    Uninstalling llvmlite-0.43.0:
      Successfully uninstalled llvmlite-0.43.0
  Attempting uninstall: pandas
    Found existing ins

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 100.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 19.6 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.3
    Uninstalling scipy-1.16.3:
      Successfully uninstalled scipy-1.16.3
  Attempting uninstall: plotly
    Found existing installation: plotly 5.24.1
    Uninstalling plotly-5.24.1:
      Successfully uninstalled plotly-5.24.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00


In [1]:
# Before running this cell, restart the runtime by selecting, "Restart runtime", from the "Runtime" menu.

# Import statements - for many scenarios, the only import needed will be `from openbb import obb`
from typing import Literal
from IPython.display import display
from IPython.display import clear_output
import ipywidgets as widgets
import pandas as pd
import pandas_ta as ta
from datetime import datetime , date
from plotly import graph_objects as go

from openbb import obb

Extensions to add: benzinga@1.5.0, bls@1.2.0, cftc@1.2.0, commodity@1.4.0, congress_gov@1.1.0, crypto@1.5.0, currency@1.5.0, derivatives@1.5.0, econdb@1.4.0, economy@1.5.0, equity@1.5.0, etf@1.5.0, federal_reserve@1.5.0, finviz@1.4.0, fixedincome@1.5.0, fmp@1.5.1, fred@1.5.0, imf@1.2.0, index@1.5.0, intrinio@1.5.0, news@1.5.0, oecd@1.5.0, openbb_charting@2.4.0, polygon@1.5.0, regulators@1.5.0, sec@1.5.0, tiingo@1.5.0, tradingeconomics@1.5.0, us_eia@1.2.0, uscongress@1.1.0, yfinance@1.5.0

Building...


## Testing cotc

In [22]:
import requests
from datetime import date
def get_historical_prices(ticker, start_date, key):

  hist_url = f'https://financialmodelingprep.com/stable/historical-price-eod/full?symbol={ticker}&apikey={key}&from=2004-01-01'

  data =  requests.get(hist_url).json()
  return [d for d in data if datetime.strptime(d['date'], '%Y-%m-%d').date() >=start_date]

from google.colab import userdata
key = userdata.get('FMP_KEY')
vix_prices = get_historical_prices('^VIX', date(2004, 7, 20), key)
# resampling prices
df = pd.DataFrame(vix_prices)
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date').sort_index()
df_weekly = df['close'].resample('W-TUE').last().dropna().reset_index()
df_weekly = df_weekly.set_index('date').sort_index()


In [24]:
cot_df = obb.regulators.cftc.cot(id='1170E1', provider='cftc').to_df()

cot_noindex = cot_df.reset_index()
cot_noindex['date'] = pd.to_datetime(cot_noindex['date'])
cot_noindex = cot_noindex.set_index('date').sort_index()


In [25]:
joined = df_weekly.join(cot_noindex, how='inner')  # or 'left', 'right', 'outer'
joined[0:2]

,close,report_week,market_and_exchange_names,cftc_contract_market_code,cftc_market_code,cftc_region_code,cftc_commodity_code,commodity,commodity_group,commodity_subgroup,...,change_in_open_interest_all,change_in_noncomm_long_all,change_in_noncomm_short_all,change_in_noncomm_spead_all,change_in_comm_long_all,change_in_comm_short_all,change_in_tot_rept_long_all,change_in_tot_rept_short,change_in_nonrept_long_all,change_in_nonrept_short_all
date,,,,,,,,,,,,,,,,,,,,,
2004-07-27,16.55,2004 Report Week 30,VIX FUTURES - CBOE FUTURES EXCHANGE,1170E1,E,CHI,117,VOLATILITY/VARIANCE INDEX,FINANCIAL INSTRUMENTS,OTHER FINANCIAL INSTRUMENTS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-08-03,16.03,2004 Report Week 31,VIX FUTURES - CBOE FUTURES EXCHANGE,1170E1,E,CHI,117,VOLATILITY/VARIANCE INDEX,FINANCIAL INSTRUMENTS,OTHER FINANCIAL INSTRUMENTS,...,400.0,-71.0,430.0,210.0,113.0,184.0,252.0,824.0,148.0,-424.0


In [21]:
LOOKBACK_WINDOW_SIZE = 156
from openbb import obb
from pprint import pprint
import pandas as pd

import pandas as pd
import numpy as np

import pandas as pd
import numpy as np

class VixSentimentCalculator:
    """
    Calculates VIX Sentiment: VIX COT Index and VIX Open Interest (OI) Ratio,
    with proper time-series alignment to COT report dates (Tuesdays).
    """

    # Define the core columns needed for VIX COT analysis
    COT_COLUMNS = {
        'report_week': 'date',
        'noncomm_positions_long_all': 'noncomm_long',
        'noncomm_positions_short_all': 'noncomm_short',
        'open_interest_all': 'open_interest',
        'commodity': 'commodity_name'
    }

    # VIX Futures CFTC Commodity Code
    VIX_CODE = '1170E1'

    def __init__(self, cot_lookback_period: int = 52 * 5, oi_lookback_period: int = 52 * 1):
        """
        Initializes the calculator.

        Args:
            cot_lookback_period (int): Weeks for COT Index historical range (e.g., 5 years).
            oi_lookback_period (int): Weeks for VIX OI sentiment ratio calculation (e.g., 1 year).
        """
        self.cot_lookback_period = cot_lookback_period
        self.oi_lookback_period = oi_lookback_period

    # --- Data Preparation ---

    def _prepare_vix_cot_data(self, cot_df: pd.DataFrame) -> pd.DataFrame:
        """Filters, renames, and calculates Net Position and VIX OI Ratio."""

        vix_cot_df = cot_df[
            cot_df['cftc_commodity_code'] == self.VIX_CODE
        ].copy()

        vix_cot_df = vix_cot_df.rename(columns=self.COT_COLUMNS)

        # 1. Standardize COT Index: date is the COT report's Tuesday
        vix_cot_df['date'] = pd.to_datetime(vix_cot_df['date'])
        vix_cot_df = vix_cot_df.set_index('date').sort_index()

        # 2. Calculate Non-Commercial Net Position
        vix_cot_df['noncomm_net'] = (
            vix_cot_df['noncomm_long'] - vix_cot_df['noncomm_short']
        )

        # 3. Calculate VIX OI Ratio (Rolling Mean / Current OI)
        vix_cot_df['vix_oi_ratio'] = (
            vix_cot_df['open_interest'].rolling(window=self.oi_lookback_period).mean() /
            vix_cot_df['open_interest']
        )

        return vix_cot_df[['noncomm_net', 'open_interest', 'vix_oi_ratio']]

    def _prepare_vix_price_data(self, vix_df: pd.DataFrame) -> pd.DataFrame:
        """
        Ensures VIX price data is indexed by date and resamples it to the
        weekly COT date (Tuesday).
        """
        # --- AMENDMENT START ---
        # Check for the correct lowercase column name
        if 'close' not in vix_df.columns:
            raise ValueError("VIX DataFrame must contain a 'close' column (lowercase).")

        # 1. Ensure Index is Datetime and Sorted
        vix_df.index = pd.to_datetime(vix_df.index.values if not isinstance(vix_df.index, pd.DatetimeIndex) else vix_df.index)
        vix_df = vix_df.sort_index()

        # 2. Resample VIX Prices to Weekly Tuesdays
        # Use the lowercase 'close' column
        vix_weekly = vix_df['close'].resample('W-TUE').last().to_frame()
        vix_weekly.columns = ['vix_close'] # Standardize output column name

        # --- AMENDMENT END ---
        return vix_weekly

    # --- Calculation ---

    def _calculate_cot_index(self, vix_cot_df: pd.DataFrame) -> pd.DataFrame:
        """Calculates the COT Index for the Non-Commercial Net Position."""

        net_position = vix_cot_df['noncomm_net']

        # Calculate the rolling maximum and minimum of the Net Position
        rolling_max = net_position.rolling(window=self.cot_lookback_period).max()
        rolling_min = net_position.rolling(window=self.cot_lookback_period).min()

        # COT Index Formula
        cot_index = ((net_position - rolling_min) / (rolling_max - rolling_min)) * 100

        vix_cot_df['vix_cot_index'] = cot_index

        return vix_cot_df

    # --- Main Method ---

    def calculate_sentiment(self, vix_df: pd.DataFrame, cot_df: pd.DataFrame) -> pd.DataFrame:
        """
        Combines VIX and COT data after resampling, calculates indices,
        and returns the merged result.
        """
        # 1. Prepare and calculate Net Position and OI Ratio from COT data
        vix_cot_data = self._prepare_vix_cot_data(cot_df)

        # 2. Prepare and resample VIX price data (uses lowercase 'close')
        vix_weekly_price = self._prepare_vix_price_data(vix_df)

        # 3. Calculate the VIX COT Index
        vix_cot_indexed = self._calculate_cot_index(vix_cot_data)

        # 4. Merge DataFrames on the date index (COT report Tuesday)
        final_df = vix_cot_indexed.merge(
            vix_weekly_price,
            left_index=True,
            right_index=True,
            how='inner'
        )

        return final_df.dropna(subset=['vix_cot_index'])

    def get_text_sentiment(self, final_df: pd.DataFrame) -> str:
        """
        Generates a text sentiment based on the latest VIX COT Index and VIX Price.
        """
        if final_df.empty:
            return "Error: No data available for sentiment calculation."

        latest = final_df.iloc[-1]
        cot_index = latest['vix_cot_index']
        vix_price = latest['vix_close']

        sentiment = ""

        # --- 1. Base Sentiment from VIX COT Index (Primary Signal) ---
        if cot_index <= 10:
            sentiment = "Extremely Bullish (VIX COT at historical low, speculators are aggressively short VIX)."
        elif cot_index <= 30:
            sentiment = "Bullish (VIX COT well below neutral, large speculators are short VIX)."
        elif cot_index >= 90:
            sentiment = "Extremely Bearish (VIX COT at historical high, speculators are heavily long VIX/hedging)."
        elif cot_index >= 70:
            sentiment = "Bearish (VIX COT well above neutral, large speculators are long VIX)."
        else:
            sentiment = "Neutral/Moderate (VIX COT positioning is balanced)."

        # --- 2. VIX Price Context (Confirmation) ---
        if vix_price > 25:
            sentiment += f" VIX Price ({vix_price:.1f}) is high, suggesting actual market stress."
        elif vix_price < 15 and "Bullish" in sentiment:
            sentiment += f" VIX Price ({vix_price:.1f}) is low, confirming complacency."
        elif vix_price < 15 and "Bearish" in sentiment:
             sentiment += f" VIX Price ({vix_price:.1f}) is low, suggesting a potential short-term reversal or false COT signal."

        return f"Latest VIX Sentiment ({latest.name.strftime('%Y-%m-%d')}): {sentiment}"




In [27]:
calculator =  VixSentimentCalculator(cot_lookback_period=52 * 5, oi_lookback_period=52 * 1)
res = calculator.calculate_sentiment(df_weekly, cot_noindex)
res


,noncomm_net,open_interest,vix_oi_ratio,vix_cot_index,vix_close
date,,,,,


Logging in to OBB

In [ ]:
# Login to OpenBB Hub to retrieve stored API keys.
# https://my.openbb.co/app/platform/pat
# https://my.openbb.co/app/platform/api-keys
from google.colab import userdata

MY_PAT = userdata.get('PAT_KEY')


obb.account.login(pat=MY_PAT)
# Setting preferences to always return a dataframe
obb.user.preferences.output_type = "dataframe"

### Defining Tickers for Analysis


In [ ]:
#obb.equity.estimates.consensus(symbol='DIS')  consensus
obb.equity.fundamental.income_growth(symbol='DIS', limit=3).T

,0,1,2
period_ending,2024-09-28,2023-09-30,2022-10-01
fiscal_period,FY,FY,FY
fiscal_year,2024,2023,2022
symbol,DIS,DIS,DIS
growth_revenue,0.027706,0.07466,0.227002
growth_cost_of_revenue,-0.008496,0.088234,0.205402
growth_gross_profit,0.099875,0.048586,0.270741
growth_gross_profit_margin,0.070224,-0.024262,0.035647
growth_operating_expenses,-0.112684,0.083747,0.167144
growth_cost_and_expenses,-0.004993,0.049119,0.194285


In [ ]:
obb.equity.fundamental.cash_growth(symbol='DIS', limit=3).T

,0,1,2
period_ending,2024-09-28,2023-09-30,2022-10-01
fiscal_period,FY,FY,FY
fiscal_year,2024,2023,2022
symbol,DIS,DIS,DIS
growth_net_income,0.70295,-0.045877,0.401025
growth_depreciation_and_amortization,-0.07059,0.039899,0.010174
growth_deferred_income_tax,0.390045,-7.73,1.16116
growth_stock_based_compensation,0.195101,0.169908,0.628333
growth_change_in_working_capital,-1.0,-0.637295,-0.777575
growth_account_receivables,-2.578212,-0.408264,4.252688


In [ ]:
# Set the output preference, if desired. The examples below use Pandas DataFrames.
from datetime import timedelta
from dateutil.relativedelta import relativedelta
TICKER = 'DIS'
TICKER2 = '^GSPC' # S&P500
SECTOR_TICKER = 'XLY'
INDUSTRY_TICKER = 'XME'
startDate = date.today() - relativedelta(years=5)

### Return Comparison

In [ ]:
desc_filters = {
        'Market Cap.': '+Small (over $300mln)',
        'Average Volume': 'Over 200K',
        #'Relative Volume': 'Over 1',
    }
fund_filters = {
    'Return on Equity': 'Positive (>0%)',
    'InstitutionalOwnership': 'Under 60%',
    'Current Ratio' :  'Over 1.5',
    'Debt/Equity'   : 'Over 0.3'
}

desc_filters.update(fund_filters)

obb.equity.screener(provider='finviz', industry='semiconductors',
                    filters_dict=desc_filters
                    )

,symbol,name,country,sector,industry,market_cap,price,change_percent,volume,price_to_earnings
0,HIMX,Himax Technologies ADR,Taiwan,Technology,Semiconductors,1.080000e+09,6.32,0.0276,1127546,13.82


In [ ]:
44import matplotlib.pyplot as plt
spy = obb.equity.price.historical(symbol = TICKER2, provider='fmp')[['close']].rename(columns={'close' : TICKER2})
dis = obb.equity.price.historical(symbol = TICKER, provider='fmp')[['close']].rename(columns={'close' : TICKER})
sector = obb.equity.price.historical(symbol = SECTOR_TICKER, provider='fmp')[['close']].rename(columns={'close' : SECTOR_TICKER})
industry = obb.equity.price.historical(symbol = INDUSTRY_TICKER, provider='fmp')[['close']].rename(columns={'close' : INDUSTRY_TICKER})

one = pd.merge(spy, dis, left_index=True, right_index=True)
two = pd.merge(one, sector,left_index=True, right_index=True)
result = pd.merge(two, industry,left_index=True, right_index=True)
histpriceconcat = result/result.iloc[0]


histpriceconcat

for i, col in enumerate([TICKER2, TICKER, SECTOR_TICKER, INDUSTRY_TICKER]):
    histpriceconcat[col].plot()

plt.title('Price Evolution Comparison')
plt.xticks(rotation=70)

legendCols = [TICKER2, TICKER, f'{SECTOR_TICKER}(Sector)', f'{INDUSTRY_TICKER}(Industry)']

plt.legend(legendCols)
plt.show()



### Overview of the company

In [ ]:
obb.equity.profile(symbol=TICKER, provider='fmp')

### PEERS


In [ ]:
obb.equity.compare.peers(symbol=TICKER)

### Fetching some ratios


### Ratios Overview

In [ ]:
liq_ratios = ['current_ratio', 'quick_ratio', 'operating_cash_flow_per_share',  'days_of_sales_outstanding' ]
solvency_ratios = ['interest_coverage', 'debt_ratio',  'debt_equity_ratio'] #missing shareholder equity ratio
profitability_ratios = ['gross_profit_margin', 'operating_profit_margin',
                        'pretax_profit_margin',
                        'net_profit_margin',
                        'return_on_assets',
                        'return_on_equity',
                        'return_on_capital_employed',
                         ]

### Quarterly

In [ ]:
annual_ratios = obb.equity.fundamental.ratios(symbol=TICKER, period="annual", limit=5)
quarterly_ratios = obb.equity.fundamental.ratios(symbol=TICKER, period="quarter", limit=12)
quarterly_ratios['key'] = quarterly_ratios.apply(lambda row: f"{row['fiscal_period']}_{row['fiscal_year']}", axis=1)
# Income statement. we need this for trajectory of fundamentals
annual_income = obb.equity.fundamental.income(symbol=TICKER, period="annual", limit=5)
quarterly_income = obb.equity.fundamental.income(symbol=TICKER, period="quarter", limit=12)
quarterly_income['key'] = quarterly_ratios.apply(lambda row: f"{row['fiscal_period']}_{row['fiscal_year']}", axis=1)

quarterly_income

### Annually

In [ ]:
annual_liquidity = annual_ratios[liq_ratios + ['fiscal_year'] ].set_index('fiscal_year')
annual_solvency = annual_ratios[solvency_ratios + ['fiscal_year'] ].set_index('fiscal_year')
annual_profitability = annual_ratios[profitability_ratios + ['fiscal_year'] ].set_index('fiscal_year')
quarterly_liquidity = quarterly_ratios[liq_ratios + ['key'] ].set_index('key')
quarterly_solvency = quarterly_ratios[solvency_ratios + ['key'] ].set_index('key')
quarterly_profitability = quarterly_ratios[profitability_ratios + ['key'] ].set_index('key')

#annual_income = annual_income.set_index('fiscal_year')
#quarterly_income = quarterly_income.set_index('key')


annual_liquidity

### Some bar charts with OBB

In [ ]:
from openbb_charting.charts.generic_charts import bar_chart

bar_chart([annual_liquidity], x='fiscal_year', y=liq_ratios, title='Liquidity Ratios annual', normalize=True).show()

###Dividends Over Time

In [ ]:
df = obb.equity.fundamental.dividends(TICKER)
df['cob'] = pd.to_datetime(df.ex_dividend_date)
df = df.set_index(df.cob)
df[df.index > startDate.strftime('%Y-%m-%d')].plot(x='cob')


### Estimates

In [ ]:
obb.equity.estimates.consensus(symbol=TICKER)

### Splits

In [ ]:
obb.equity.fundamental.historical_splits(TICKER)

### Financial Statement Growth

In [ ]:
obb.equity.fundamental.balance_growth(symbol=TICKER, limit=10)

In [ ]:
obb.equity.fundamental.income_growth(symbol=TICKER, limit=10)

In [ ]:
obb.equity.fundamental.cash_growth(symbol=TICKER, limit=10)

### Insider Tradings

In [ ]:
obb.equity.ownership.insider_trading(symbol=TICKER, limit=500)

### 13F  - not Disney related -

In [ ]:
obb.equity.discovery.filings(limit=100, form_type='13F')

### Equity discovery

### Most Active

In [ ]:
obb.equity.discovery.active(sort="desc")

### Gainers

In [ ]:
obb.equity.discovery.gainers(sort="desc")

### Undervalued large caps

In [ ]:
obb.equity.discovery.undervalued_large_caps(sort="desc")

### Gainers

In [ ]:
obb.equity.discovery.gainers(limit=5)

In [ ]:
obb.equity.discovery.undervalued_growth(sort="desc")